In [1]:
import pandas as pd
from datetime import timedelta, datetime, timezone
import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.proportion import proportions_ztest

In [2]:
df1=pd.read_csv(r'L:\Thesis\Exploratory-Analysis-of-Developer-Sentiment-on-Open-Source-Projects\Dataset\commits.csv')

In [4]:
len(df1['repo'].unique())

1628

# RQ-2


In [42]:
df2=pd.read_csv(r'L:\Thesis\Exploratory-Analysis-of-Developer-Sentiment-on-Open-Source-Projects\Dataset\commits.csv')

In [49]:
df2['message_sentiment'].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [50]:
df2['message_sentiment'].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [51]:
night_count = (df2['part_of_day_commit'] == 'Night').sum()

# Subset the data for negative sentiment
negative_sentiment_data = df2[df2['message_sentiment'] == 'negative']

# Create subsets for irregular and regular times
irregular_times_data = negative_sentiment_data[negative_sentiment_data['part_of_day_commit'] == 'Night']
regular_times_data = negative_sentiment_data[~(negative_sentiment_data['part_of_day_commit'] == 'Night')]

# Calculate proportions
if len(negative_sentiment_data) > 0:  # Check if there are any negative sentiment messages
    irregular_proportion = len(irregular_times_data) / len(negative_sentiment_data)
    regular_proportion = len(regular_times_data) / len(negative_sentiment_data)

    # Perform z-test
    z_stat, p_value = proportions_ztest([len(irregular_times_data), len(regular_times_data)], 
                                         [night_count, len(negative_sentiment_data) - night_count])

    # Output the results
    print("Proportion of negative sentiment at irregular times:", irregular_proportion)
    print("Proportion of negative sentiment at regular times:", regular_proportion)
    print("Z-score:", z_stat)
    print("p-value:", p_value)

    # Determine significance
    alpha = 0.05
    if p_value < alpha:
        print("Reject the null hypothesis: There is a significant difference in the proportion of negative sentiment between irregular and regular times.")
    else:
        print("Fail to reject the null hypothesis: There is no significant difference in the proportion of negative sentiment between irregular and regular times.")
else:
    print("There are no negative sentiment messages in the dataset.")

Proportion of negative sentiment at irregular times: 0.13821345455747053
Proportion of negative sentiment at regular times: 0.8617865454425294
Z-score: -inf
p-value: 0.0
Reject the null hypothesis: There is a significant difference in the proportion of negative sentiment between irregular and regular times.


d:\Anaconda_envs\envs\myenv\lib\site-packages\statsmodels\stats\weightstats.py:792: RuntimeWarning: divide by zero encountered in double_scalars
  zstat = value / std


# RQ3

In [53]:
df1=pd.read_csv(r'L:\Thesis\Exploratory-Analysis-of-Developer-Sentiment-on-Open-Source-Projects\Dataset\RQ3.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_16576\3685222560.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df1=pd.read_csv(r'L:\Thesis\Exploratory-Analysis-of-Developer-Sentiment-on-Open-Source-Projects\Dataset\RQ3.csv')


In [54]:
df1.head()

,issuetype,summary,summary_sentiment
0,Bug,1-800-790-9186 Epson Tech Support TelePhone Nu...,neutral
1,Bug,pogo games customer service phone number 1855 ...,neutral
2,Bug,KUND pogo games customer service phone number ...,neutral
3,Bug,"phone number 1855 800 8493, Pogo games tech Su...",neutral
4,Bug,"call, 1855 800 8493 for all type help by Pogo ...",neutral


In [56]:
negative_sentiment_data = df1[df1['summary_sentiment'] == 'negative']

# Create subsets for bug and non-bug issues
bug_data = negative_sentiment_data[negative_sentiment_data['issuetype'] == 'bug']
non_bug_data = negative_sentiment_data[negative_sentiment_data['issuetype'] != 'bug']

# Calculate proportions
bug_proportion = len(bug_data) / len(negative_sentiment_data)
non_bug_proportion = len(non_bug_data) / len(negative_sentiment_data)

# Perform z-test
z_stat, p_value = proportions_ztest([len(bug_data), len(non_bug_data)], 
                                          [len(negative_sentiment_data), len(negative_sentiment_data)])

# Output the results
print("Proportion of negative sentiment in bug issues:", bug_proportion)
print("Proportion of negative sentiment in non-bug issues:", non_bug_proportion)
print("Z-score:", z_stat)
print("p-value:", p_value)

# Determine significance
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference in the proportion of negative sentiment between bug and non-bug issues.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference in the proportion of negative sentiment between bug and non-bug issues.")

Proportion of negative sentiment in bug issues: 0.0
Proportion of negative sentiment in non-bug issues: 1.0
Z-score: -1274.3782797897961
p-value: 0.0
Reject the null hypothesis: There is a significant difference in the proportion of negative sentiment between bug and non-bug issues.


In [63]:
def compute_negative_sentiment_score(df):
    negative_sentiment_data = df[df['summary_sentiment'] == 'negative']
    return negative_sentiment_data['sentiment_score'].mean()  # Assuming sentiment_score is the column representing sentiment score

# Compute the negative sentiment score for bug and non-bug issues

bug_negative_score = compute_negative_sentiment_score(df1[df1['issuetype'] == 'Bug'])
non_bug_negative_score = compute_negative_sentiment_score(df1[df1['issuetype'] != 'Bug'])

# Plotting
plt.figure(figsize=(8, 6))
plt.bar(['Bug', 'Non-Bug'], [bug_negative_score, non_bug_negative_score])
plt.title('Average Sentiment Score for Negative Sentiment in Bug and Non-Bug Issues')
plt.ylabel('Average Sentiment Score')
plt.xlabel('Issue Type')
plt.grid(True)
plt.show()

KeyError: 'sentiment_score'

# RQ1

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [2]:


df = pd.read_csv(r'L:\Thesis\Exploratory-Analysis-of-Developer-Sentiment-on-Open-Source-Projects\Dataset\commits.csv')


In [3]:
repositories_of_interest = ['gecko-dev', 'hadoop', 'tomcat', 'lucene-solr', 'camel']

In [4]:
df=df[df['repo'].isin(repositories_of_interest)]

In [5]:
len(df)

1016723

In [12]:


# Calculate the total number of days in the project
df['commit_time'] = pd.to_datetime(df['commit_time'])
D = (df['commit_time'].max() - df['commit_time'].min()).days + 1

# Calculate commits per day for each developer
df['commit_date'] = df['commit_time'].dt.date
commits_per_day = df.groupby(['committer', 'commit_date']).size().reset_index(name='commits')

# Calculate the regularity index R for each developer
def calculate_regularity_index(committer_data):
    di = committer_data['commit_date'].nunique()
    C = committer_data['commits'].sum()
    avg_commits_per_day = C / di
    commit_sd = np.sqrt((1/di) * np.sum((committer_data['commits'] - avg_commits_per_day)**2))
    R = (di / D) * (1 / (commit_sd + 1))
    return R, di, C

regularity_indices = commits_per_day.groupby('committer').apply(calculate_regularity_index).reset_index()
regularity_indices.columns = ['committer', 'regularity_data']
regularity_indices[['R', 'di', 'C']] = pd.DataFrame(regularity_indices['regularity_data'].tolist(), index=regularity_indices.index)
regularity_indices.drop(columns='regularity_data', inplace=True)

# Classify committers into regular, average, and irregular
Q1 = regularity_indices['R'].quantile(0.25)
Q3 = regularity_indices['R'].quantile(0.75)
IQR = Q3 - Q1

regularity_indices['category'] = 'average'
regularity_indices.loc[regularity_indices['R'] <= Q1, 'category'] = 'irregular'
regularity_indices.loc[regularity_indices['R'] >= Q3, 'category'] = 'regular'

# Merge with the original dataframe to get sentiments
df = df.merge(regularity_indices[['committer', 'category']], on='committer', how='left')

# Calculate the proportion of negative sentiment messages
negative_sentiment_counts = df[df['message_sentiment'] == 'negative'].groupby('category').size()
total_counts = df.groupby('category').size()
proportion_negative = negative_sentiment_counts/ total_counts

# Plotting

# Distribution of Regularity Index
plt.figure(figsize=(10, 6))
sns.histplot(regularity_indices['R'], kde=True, bins=30)
plt.title('Distribution of Regularity Index')
plt.xlabel('Regularity Index (R)')
plt.ylabel('Frequency')
plt.show()

# Boxplot of Regularity Index
plt.figure(figsize=(10, 6))
sns.boxplot(x=regularity_indices['R'])
plt.title('Boxplot of Regularity Index')
plt.xlabel('Regularity Index (R)')
plt.show()

# Bar Plot of Sentiment Proportion
proportion_negative.plot(kind='bar', color=['red', 'green', 'blue'])
plt.title('Proportion of negative Sentiment Messages by Committer Category')
plt.xlabel('Committer Category')
plt.ylabel('Proportion of negative Sentiment Messages')
plt.show()

# Pie Chart of Sentiment Distribution
for category in df['category'].unique():
    plt.figure(figsize=(6, 6))
    df[df['category'] == category]['message_sentiment'].value_counts().plot(kind='pie', autopct='%1.1f%%')
    plt.title(f'Sentiment Distribution for {category.capitalize()} Committers')
    plt.ylabel('')
    plt.show()



C:\Users\User\AppData\Local\Temp\ipykernel_13016\1925285737.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  regularity_indices = commits_per_day.groupby('committer').apply(calculate_regularity_index).reset_index()


MergeError: Passing 'suffixes' which cause duplicate columns {'category_x'} is not allowed.

In [9]:
from statsmodels.stats.proportion import proportions_ztest

# Calculate the number of negative sentiments for irregular and regular committers
negative_sentiments_irregular = df[(df['category'] == 'irregular') & (df['message_sentiment'] == 'negative')].shape[0]
negative_sentiments_regular = df[(df['category'] == 'regular') & (df['message_sentiment'] == 'negative')].shape[0]

# Calculate the total number of sentiments for irregular and regular committers
total_sentiments_irregular = df[df['category'] == 'irregular'].shape[0]
total_sentiments_regular = df[df['category'] == 'regular'].shape[0]

# Proportions of negative sentiments
proportion_negative_irregular = negative_sentiments_irregular / total_sentiments_irregular
proportion_negative_regular = negative_sentiments_regular / total_sentiments_regular

# Perform two-proportion z-test
count = np.array([negative_sentiments_irregular, negative_sentiments_regular])
nobs = np.array([total_sentiments_irregular, total_sentiments_regular])

stat, p_value = proportions_ztest(count, nobs, alternative='larger')

print(f"Proportion of negative sentiment messages for irregular committers: {proportion_negative_irregular:.2f}")
print(f"Proportion of negative sentiment messages for regular committers: {proportion_negative_regular:.2f}")
print(f"Z-test statistic: {stat:.2f}")
print(f"p-value: {p_value:.4f}")

# Conclusion
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: Irregular committers have a higher proportion of negative sentiment messages than regular committers.")
else:
    print("Fail to reject the null hypothesis: No significant evidence that irregular committers have a higher proportion of negative sentiment messages than regular committers.")


Proportion of negative sentiment messages for irregular committers: 0.02
Proportion of negative sentiment messages for regular committers: 0.03
Z-test statistic: -5.42
p-value: 1.0000
Fail to reject the null hypothesis: No significant evidence that irregular committers have a higher proportion of negative sentiment messages than regular committers.


In [ ]:
repo=['gecko-dev','hadoop','tomcat','lucene-solr','camel']

# RQ4

In [13]:
df=pd.read_csv(r'L:\Thesis\Exploratory-Analysis-of-Developer-Sentiment-on-Open-Source-Projects\Dataset\RQ4.csv')

In [14]:
df.head()

,source,product,issue_id,comment_id,count,author_key,author_tz,update_author_key,update_author_tz,created,updated,index,text,text_sentiment
0,apache,AAR,12963270.0,15261151.0,NaN,14e647a68666823ae46697258899ac0b0a681de6,Etc/UTC,14e647a68666823ae46697258899ac0b0a681de6,Etc/UTC,2016-04-27 23:04:37+00:00,2016-04-27 23:05:24+00:00,2,"Watch The Ultimate Fighter Season 23, Episode ...",neutral
1,apache,AAR,12963426.0,15261824.0,NaN,dbea2a0f9161664766394eaeae89d872577bc764,Europe/Berlin,dbea2a0f9161664766394eaeae89d872577bc764,Europe/Berlin,2016-04-28 09:16:51+00:00,2016-04-28 09:16:51+00:00,3,faqegggf,neutral
2,apache,AAR,12963426.0,15262448.0,NaN,dbea2a0f9161664766394eaeae89d872577bc764,Europe/Berlin,dbea2a0f9161664766394eaeae89d872577bc764,Europe/Berlin,2016-04-28 16:30:42+00:00,2016-04-28 16:30:42+00:00,4,geadggdgdg,neutral
3,apache,AAR,12963426.0,15262452.0,NaN,dbea2a0f9161664766394eaeae89d872577bc764,Europe/Berlin,dbea2a0f9161664766394eaeae89d872577bc764,Europe/Berlin,2016-04-28 16:31:30+00:00,2016-04-28 16:31:30+00:00,5,gaeggg,negative
4,apache,AAR,12963426.0,15262453.0,NaN,dbea2a0f9161664766394eaeae89d872577bc764,Europe/Berlin,dbea2a0f9161664766394eaeae89d872577bc764,Europe/Berlin,2016-04-28 16:31:50+00:00,2016-04-28 16:31:50+00:00,6,fegf,neutral


In [37]:
df1=pd.read_csv(r'L:\Thesis\Exploratory-Analysis-of-Developer-Sentiment-on-Open-Source-Projects\Dataset\issues.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_13016\3740768291.py:1: DtypeWarning: Columns (3,6,8,9,10,12,14,18,19,20,22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df1=pd.read_csv(r'L:\Thesis\Exploratory-Analysis-of-Developer-Sentiment-on-Open-Source-Projects\Dataset\issues.csv')


In [34]:
df1=df1[['severity','summary_sentiment']]

In [35]:
df1['severity'].unique()

array(['normal', 'enhancement', 'major', 'critical', 'minor', 'trivial',
       'blocker'], dtype=object)

In [38]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

# Load the dataset




# Calculate the counts needed for the Z-test
major_negative_count = df1[(df['severity'] == 'major') & (df1['summary_sentiment'] == 'negative')].shape[0]
major_total_count = df1[df1['severity'] == 'major'].shape[0]

other_negative_count = df1[(df1['severity'] != 'major') & (df1['summary_sentiment'] == 'negative')].shape[0]
other_total_count = df1[df1['severity'] != 'major'].shape[0]

# Perform the Z-test for proportions
count = np.array([major_negative_count, other_negative_count])
nobs = np.array([major_total_count, other_total_count])

stat, p_value = proportions_ztest(count, nobs)

print(f"Z-test statistic: {stat}")
print(f"P-value: {p_value}")

if p_value < 0.05:
    print("There is a significant difference in the proportion of negative sentiment for 'major' severity compared to other severities.")
else:
    print("There is no significant difference in the proportion of negative sentiment for 'major' severity compared to other severities.")


KeyError: 'severity'